# Neo4j Dev
Purpose is to build a prototype Neo4j instance out.<br>
By: Jonathan Lo<br>
Date: 11/12/23

## Overhead

In [1]:
# Imports
from json import load
from neo4j import GraphDatabase
from tqdm.notebook import tqdm

In [2]:
# Import DB details from config
config = load(open('config.json', 'r'))

# Load interactive video data
iv_data = load(open('events.json', 'r'))

In [3]:
# DB Connection
URI = config['dbURI']
AUTH = (config['dbAuth']['user'], config['dbAuth']['pass'])

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

## Analysis

In [4]:
# Preproccess
events_list = list(iv_data.values())

In [5]:
def insert_data(driver, data):
    with driver.session() as session:
        for event, details in tqdm(data.items()):
            # Check for null values before inserting
            name = details['name'] if details['name'] else ""
            title = details['title'] if details['title'] else ""
            start = details['start'] if details['start'] else ""
            end = details['end'] if details['end'] else ""
            desc = details['desc']['short'] if details['desc']['short'] else ""
            vid_link = details['vid_link'] if details['vid_link'] else ""
            
            # Event Query
            e_query = """MERGE (
                e:Event {
                    name: $name,
                    title: $title,
                    start: $start,
                    end: $end, desc:
                    $desc, vid_link:
                    $vid_link
                })
            """
            session.run(
                e_query,
                name=name,
                title=title,
                start=start,
                end=end,
                desc=desc,
                vid_link=vid_link
            )
            
            # Tag Qeury
            t_query = """
                MERGE (t:Tag {name: $name}) MERGE (e)-[:HAS_TAG]->(t)
            """
            for tag in details['tags']:
                session.run(t_query, name=tag)
                
            # Instructor Query
            i_query = """
                MERGE (i:Instructor {label: $label, title: $title, bio: $bio}) MERGE (e)-[:HAS_INSTRUCTOR]->(i)
            """
            if details['instr']['label']:
                label = details['instr']['label'] if details['instr']['label'] else ""
                instr_title = details['instr']['title'] if details['instr']['title'] else ""
                bio = details['instr']['bio'] if details['instr']['bio'] else ""
                session.run(
                    i_query,
                    label=label,
                    title=instr_title,
                    bio=bio
                )
                
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    insert_data(driver, iv_data)

  0%|          | 0/74 [00:00<?, ?it/s]